In [275]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
import tensorflow as tf
import gradio as gr

In [276]:
leafly = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/leafly_strain_data.csv')
leafly.head()

,name,img_url,type,thc_level,most_common_terpene,description,relaxed,happy,euphoric,uplifted,...,fibromyalgia,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus
0,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
1,Wedding Cake,https://images.leafly.com/flower-images/weddin...,Hybrid,22%,Limonene,Wedding Cake is a potent indica-hybrid marijua...,60%,50%,41%,35%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
2,Runtz,https://images.leafly.com/flower-images/runtz-...,Hybrid,19%,Limonene,Runtz is a rare hybrid marijuana strain by Coo...,47%,62%,48%,43%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
3,Gelato,https://images.leafly.com/flower-images/gelato...,Hybrid,17%,Caryophyllene,"Gelato, aka Larry Bird, is a hybrid marijuana ...",54%,49%,46%,37%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
4,Blue Dream,https://images.leafly.com/flower-images/blue-d...,Hybrid,18%,Myrcene,Blue Dream is a sativa-dominant hybrid marijua...,48%,57%,52%,48%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%


In [277]:
leafly.columns

Index(['name', 'img_url', 'type', 'thc_level', 'most_common_terpene',
       'description', 'relaxed', 'happy', 'euphoric', 'uplifted', 'sleepy',
       'dry_mouth', 'dry_eyes', 'dizzy', 'paranoid', 'anxious', 'stress',
       'pain', 'depression', 'anxiety', 'insomnia', 'hungry', 'talkative',
       'headache', 'ptsd', 'creative', 'energetic', 'fatigue', 'focused',
       'giggly', 'lack_of_appetite', 'nausea', 'headaches', 'bipolar_disorder',
       'cancer', 'tingly', 'cramps', 'aroused', 'gastrointestinal_disorder',
       'inflammation', 'muscle_spasms', 'eye_pressure', 'migraines', 'asthma',
       'anorexia', 'arthritis', 'add/adhd', 'muscular_dystrophy',
       'hypertension', 'glaucoma', 'pms', 'seizures', 'spasticity',
       'spinal_cord_injury', 'fibromyalgia', 'crohn's_disease',
       'phantom_limb_pain', 'epilepsy', 'multiple_sclerosis', 'parkinson's',
       'tourette's_syndrome', 'alzheimer's', 'hiv/aids', 'tinnitus'],
      dtype='object')

In [278]:
lab_results = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/results.csv')
lab_results.head()

C:\Users\juhyn\AppData\Local\Temp\ipykernel_21148\1654712513.py:1: DtypeWarning: Columns (2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_results = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/results.csv')


,Database Identifier,Database Name,Test Result UID,Sample Name,Sample Type,Receipt Time,Test Time,Post Time,Provider,cis-Nerolidol,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,analytical360,Analytical 360,NaN,BHO Blackberry 22,Archived,NaN,NaN,2013-08-19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.08,NaN,NaN,NaN
1,analytical360,Analytical 360,NaN,BHO The Sour Bud,Archived,NaN,NaN,2013-08-19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.75,0.34,NaN,NaN
2,analytical360,Analytical 360,NaN,Northern Lights,Archived,NaN,NaN,2013-08-19,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.05,NaN,NaN
3,analytical360,Analytical 360,NaN,Blueberry Kush,Archived,NaN,NaN,2013-08-19,3.0,NaN,...,NaN,0.04,NaN,NaN,NaN,NaN,0.25,0.19,NaN,NaN
4,analytical360,Analytical 360,NaN,White Dawg,Archived,NaN,NaN,2013-08-19,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.12,0.11,NaN,NaN


In [279]:
lab_results.columns

Index(['Database Identifier', 'Database Name', 'Test Result UID',
       'Sample Name', 'Sample Type', 'Receipt Time', 'Test Time', 'Post Time',
       'Provider', 'cis-Nerolidol', 'trans-Nerolidol', 'trans-Nerolidol 1',
       'trans-Nerolidol 2', 'trans-Ocimene', '3-Carene', 'Camphene',
       'Caryophyllene Oxide', 'Eucalyptol', 'Geraniol', 'Guaiol', 'Isopulegol',
       'Linalool', 'Ocimene', 'Terpinolene', 'alpha-Bisabolol',
       'alpha-Humulene', 'alpha-Pinene', 'alpha-Terpinene',
       'beta-Caryophyllene', 'beta-Myrcene', 'beta-Ocimene', 'beta-Pinene',
       'delta-Limonene', 'gamma-Terpinene', 'p-Cymene', 'delta-9 THC-A',
       'delta-9 THC', 'delta-8 THC', 'THC-A', 'THCV', 'CBN', 'CBD-A', 'CBD',
       'CBDV', 'CBDV-A', 'delta-9 CBG-A', 'delta-9 CBG', 'CBC',
       'Moisture Content'],
      dtype='object')

In [280]:
# copying  lab_results dataframe
lab_results_copy = lab_results.copy()

# dropping columns from lab_results
lab_results_copy = lab_results_copy.drop(['Receipt Time', 'Test Time', 'Post Time', 'Provider'], axis = 1)

lab_results_copy

,Database Identifier,Database Name,Test Result UID,Sample Name,Sample Type,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,analytical360,Analytical 360,NaN,BHO Blackberry 22,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.08,NaN,NaN,NaN
1,analytical360,Analytical 360,NaN,BHO The Sour Bud,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.75,0.34,NaN,NaN
2,analytical360,Analytical 360,NaN,Northern Lights,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.66,0.05,NaN,NaN
3,analytical360,Analytical 360,NaN,Blueberry Kush,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,0.04,NaN,NaN,NaN,NaN,0.25,0.19,NaN,NaN
4,analytical360,Analytical 360,NaN,White Dawg,Archived,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.12,0.11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43013,sclabs,SC Labs,NaN,Dream Tip THCa Primary,"Concentrate, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.0,0.0,0.32,0.00,0.00,NaN
43014,sclabs,SC Labs,NaN,Cornbread Shatter Primary,"Concentrate, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.16,0.00,0.0,0.0,3.11,0.26,0.00,NaN
43015,sclabs,SC Labs,NaN,Sample 1,"Flower, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.02,0.00,0.0,0.0,0.33,0.11,0.00,NaN
43016,sclabs,SC Labs,NaN,Sample 4,"Flower, Inhalable",NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.04,0.00,0.0,0.0,0.69,0.12,0.00,NaN


In [281]:
matching_values = leafly['name'][leafly['name'].isin(lab_results_copy['Sample Name'])]
print(matching_values)

0                GG4
1       Wedding Cake
3             Gelato
4         Blue Dream
5       Purple Punch
            ...     
4624        Good Day
4647      Stone Free
4668     Dirty Widow
4726        Deadwood
4740     Walrus Kush
Name: name, Length: 1199, dtype: object


In [282]:
# merged leafly and lab results together for prototype
merged_df = pd.merge(leafly, lab_results_copy, left_on = 'name', right_on = 'Sample Name', how = 'inner')

merged_df.head()

,name,img_url,type,thc_level,most_common_terpene,description,relaxed,happy,euphoric,uplifted,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,NaN,0.08,0.07,NaN,NaN,0.36,NaN,0.02,NaN
1,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,NaN,0.17,0.26,NaN,NaN,0.91,NaN,0.05,NaN
2,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,0.04,0.12,0.40,NaN,NaN,1.49,NaN,0.42,NaN
3,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,NaN,NaN,0.04,0.04,NaN,NaN,0.32,NaN,NaN,NaN
4,GG4,https://images.leafly.com/flower-images/gg-4.jpg,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,...,0.01,0.01,0.01,0.01,NaN,NaN,0.46,0.07,0.04,10.19


In [283]:
merged_df = merged_df.drop(['img_url'], axis=1)
merged_df

,name,type,thc_level,most_common_terpene,description,relaxed,happy,euphoric,uplifted,sleepy,...,THCV,CBN,CBD-A,CBD,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content
0,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,NaN,0.08,0.07,NaN,NaN,0.36,NaN,0.02,NaN
1,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,NaN,0.17,0.26,NaN,NaN,0.91,NaN,0.05,NaN
2,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,0.04,0.12,0.40,NaN,NaN,1.49,NaN,0.42,NaN
3,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,NaN,NaN,0.04,0.04,NaN,NaN,0.32,NaN,NaN,NaN
4,GG4,Hybrid,20%,Caryophyllene,"Original Glue (GG4), developed by GG Strains, ...",66%,54%,51%,39%,27%,...,0.01,0.01,0.01,0.01,NaN,NaN,0.46,0.07,0.04,10.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10207,Deadwood,Indica,NaN,NaN,Deadwood is potent indica cross between Bubba ...,80%,40%,0%,20%,40%,...,NaN,NaN,0.07,NaN,NaN,NaN,NaN,0.05,0.02,NaN
10208,Deadwood,Indica,NaN,NaN,Deadwood is potent indica cross between Bubba ...,80%,40%,0%,20%,40%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,NaN,NaN
10209,Deadwood,Indica,NaN,NaN,Deadwood is potent indica cross between Bubba ...,80%,40%,0%,20%,40%,...,NaN,0.05,0.07,0.07,NaN,NaN,NaN,0.16,NaN,NaN
10210,Walrus Kush,Hybrid,NaN,NaN,"While normally difficult to ignore, this Walru...",0%,80%,0%,40%,0%,...,NaN,NaN,0.05,0.07,NaN,NaN,0.24,NaN,0.02,NaN


In [284]:
# filling all null values with an empty string
merged_df = merged_df.fillna('')
print(merged_df.isnull().sum())

name                   0
type                   0
thc_level              0
most_common_terpene    0
description            0
                      ..
CBDV-A                 0
delta-9 CBG-A          0
delta-9 CBG            0
CBC                    0
Moisture Content       0
Length: 108, dtype: int64


In [285]:
# reading in the final cleaned data into a dataframe to use on the final project
final_data = pd.read_csv('C:/Users/juhyn/OneDrive/Desktop/group-project-3/data/cleaned_joined_data.csv')

final_data

,Sample Name,cis-Nerolidol,trans-Nerolidol,trans-Nerolidol 1,trans-Nerolidol 2,trans-Ocimene,3-Carene,Camphene,Caryophyllene Oxide,Eucalyptol,...,fibromyalgia,crohn's_disease,phantom_limb_pain,epilepsy,multiple_sclerosis,parkinson's,tourette's_syndrome,alzheimer's,hiv/aids,tinnitus
0,22,0.0,0.0,0.0,0.0,0.0,0.00,0.040000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24k Gold,0.0,0.0,0.0,0.0,0.0,0.00,0.011000,0.003000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3 Kings,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.533333,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3X Crazy,0.0,0.0,0.0,0.0,0.0,0.00,0.003000,0.004000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,501st OG,0.0,0.0,0.0,0.0,0.0,0.00,0.006636,0.057818,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,Zelly’s Gift,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1107,Zeus OG,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1108,Zkittlez,0.0,0.0,0.0,0.0,0.0,0.00,0.010000,0.134000,0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1109,Zookies,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.022000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [286]:
final_data.columns

Index(['Sample Name', 'cis-Nerolidol', 'trans-Nerolidol', 'trans-Nerolidol 1',
       'trans-Nerolidol 2', 'trans-Ocimene', '3-Carene', 'Camphene',
       'Caryophyllene Oxide', 'Eucalyptol',
       ...
       'fibromyalgia', 'crohn's_disease', 'phantom_limb_pain', 'epilepsy',
       'multiple_sclerosis', 'parkinson's', 'tourette's_syndrome',
       'alzheimer's', 'hiv/aids', 'tinnitus'],
      dtype='object', length=103)

In [287]:
# setting up VADER 
#analyze user input and sentiment scores
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# initializing VADER
analyzer = SentimentIntensityAnalyzer()

In [288]:
# analyzing the sentiment in the description column

def analyze_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# applying function to description column
final_data['compound score'] = final_data['description'].apply(lambda x: analyze_sentiment(x) if pd.notnull(x) else 0)

final_data[['Sample Name', 'description', 'compound score']]

,Sample Name,description,compound score
0,22,22 is a Jack Herer cross from Cali Connection....,0.6124
1,24k Gold,24k Gold is a hybrid marijuana strain made by ...,0.9042
2,3 Kings,"The 3 Kings marijuana strain, a holy trinity o...",-0.1280
3,3X Crazy,3X Crazy is an indica marijuana strain made by...,-0.6486
4,501st OG,"501st OG, bred by Rare Dankness, is an indica-...",0.7845
...,...,...,...
1106,Zelly’s Gift,Zelly’s Gift by Dragonfly Earth Medicine is a ...,0.9782
1107,Zeus OG,Zeus OG is a hybrid cross between Pineapple OG...,0.6369
1108,Zkittlez,Zkittlez is an indica-dominant mix of Grape Ap...,0.9565
1109,Zookies,Zookies is a hyrbid marijuana strain made by c...,0.9476


In [296]:
# loading hugging face sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# encoding the description column
final_data['embeddings'] = final_data['description'].apply(
    lambda x: model.encode(x, convert_to_tensor=True).cpu().numpy() if pd.notnull(x) else None
)
final_data[['description', 'embeddings']]

,description,embeddings
0,22 is a Jack Herer cross from Cali Connection....,"[0.018934406, 0.017529888, -0.00294363, 0.0294..."
1,24k Gold is a hybrid marijuana strain made by ...,"[-0.004545007, 0.009160097, -0.018976457, 0.02..."
2,"The 3 Kings marijuana strain, a holy trinity o...","[-0.031057846, 0.015036737, -0.065898985, -0.0..."
3,3X Crazy is an indica marijuana strain made by...,"[0.03299078, -0.057355344, -0.03161307, -0.072..."
4,"501st OG, bred by Rare Dankness, is an indica-...","[-0.07843714, 0.05048563, 0.044077866, -0.0300..."
...,...,...
1106,Zelly’s Gift by Dragonfly Earth Medicine is a ...,"[-0.067929655, 0.07461036, -0.007651428, 0.041..."
1107,Zeus OG is a hybrid cross between Pineapple OG...,"[-0.059249137, 0.11663607, -0.04813791, 0.0041..."
1108,Zkittlez is an indica-dominant mix of Grape Ap...,"[-0.035971727, 0.053957053, -0.0020418214, 0.0..."
1109,Zookies is a hyrbid marijuana strain made by c...,"[0.029411908, 0.06517078, 0.025467739, -0.0186..."


In [298]:
# filtering out invalid embeddings and and dropping embeddings that are invalid
embedding_dim = model.get_sentence_embedding_dimension()
final_data = final_data[final_data['embeddings'].apply(lambda x: x is not None and x.shape == (embedding_dim,))]

In [299]:
final_data.columns

Index(['Sample Name', 'cis-Nerolidol', 'trans-Nerolidol', 'trans-Nerolidol 1',
       'trans-Nerolidol 2', 'trans-Ocimene', '3-Carene', 'Camphene',
       'Caryophyllene Oxide', 'Eucalyptol',
       ...
       'phantom_limb_pain', 'epilepsy', 'multiple_sclerosis', 'parkinson's',
       'tourette's_syndrome', 'alzheimer's', 'hiv/aids', 'tinnitus',
       'compound score', 'embeddings'],
      dtype='object', length=105)

In [300]:
# adding Langchain to handle user input
def get_dynamic_input(user_input):
    prompt = ChatPromptTemplate.from_template(
        "User wants recommendation based on: {user_input}, reply with a suitable recommendation"
    )
    return prompt.format(user_input=user_input)

In [303]:
# recommendation function
def recommend_terpene(user_input, top_n=5):
    # Generating user embedding from the input text
    user_embedding = model.encode(user_input, convert_to_tensor=True).cpu().numpy()

    # Calculating the similarities between the user inputs and the embeddings
    similarities = np.inner(user_embedding, np.vstack(final_data['embeddings']))

    # Ranking recommendations
    recommended_indices = similarities.argsort()[-top_n:][::-1]

    recommendations = final_data.iloc[recommended_indices][['Sample Name', 'description', 'compound score']]

    return recommendations

In [304]:
# gradio 
def gradio_interface(user_input):
    return recommend_terpene(user_input)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter a description of the desired terpene"),
    outputs=gr.Dataframe(label="Recommended Terpenes")
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.


In [354]:
# Gradio

def gradio_interface(user_input):
    return recommend_terpene(user_input)

# customizing gradio interface with a built-in theme (soft)
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Enter a description of the desired terpene", placeholder="Type here...", lines=3)
            submit_button = gr.Button("Get Recommendations", variant="primary")
            clear_button = gr.Button("Clear")
        with gr.Column():
            output = gr.Dataframe(headers=["Sample Name", "Description", "Compound Score"], label="Recommended Terpenes")
    
    submit_button.click(gradio_interface, inputs=user_input, outputs=output)

demo.launch()


* Running on local URL:  http://127.0.0.1:7941

To create a public link, set `share=True` in `launch()`.
